# LBPP Replicate

In [8]:
from human_eval.data import write_jsonl, read_problems, LBPP_TEST
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import time, datetime
from langchain_community.llms import Replicate
import json
from typing import Iterable, Dict
import gzip

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def remove_last_triple_backticks(text: str) -> str:
    """
    Removes everything starting from the last triple backticks (` ``` `) to the end.

    :param text: The input text as a string.
    :return: A string with all text removed after the last set of triple backticks
    """
    s = "```"
    idx = text.rfind( s )
    return text[:idx]

In [3]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def read_problems(evalset_file: str) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}

In [4]:
def is_full_function(input_str: str) -> str:
    '''
    Determine if input string is a Python function
    using simple heuristics (all that's needed)
    '''
    lines = input_str.splitlines()
    for line in lines:
        if line.startswith('def '):
            return True
    return False


def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if 'def ' in text and s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [5]:
def read_jsonl(file_path):
    '''
    Read a JSONL file.
    Parameters:
        file_path (str): The path to the JSONL file.
    Returns:
        List[dict]: A list of dictionaries representing the JSON objects.
    '''
    data = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data

In [6]:
tasks = read_problems(LBPP_TEST)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: lbpp/0 <class 'str'>

task_id:
lbpp/0

test:
import numpy as np

ar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected_output = np.array([[1, 2, 3, 2.0, 0.81649658], [4, 5, 6, 5.0, 0.81649658], [7, 8, 9, 8.0, 0.81649658]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1], [1], [2]])
expected_output = np.array([[1, 1, 0.0], [1, 1, 0.0], [2, 2, 0.0]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1, 2, 3, 4, 5]])
expected_output = np.array([[1, 2, 3, 4, 5, 3.0, 1.41421356]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)

prompt:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You ma

In [22]:
solar_10b = Replicate(
    model        = 'nateraw/nous-hermes-2-solar-10.7b:1e918ab6ffd5872c21fba21a511f344fd12ac0edff6302c9cd260395c7707ff4',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

phixtral_7b = Replicate(
    model        = 'lucataco/phixtral-2x2_8:25d7b93bb0ec9e8dd94fcc69adc786759243a5628ba5574bd9609d6abafe57cf',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

mamba = Replicate(
    model        = 'adirik/mamba-2.8b:864ed9237f28ff2e8eb55ffc07e9f0d7857888cacb9c32a92081f24f49a861db',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

gemma_7b = Replicate(
    model        = 'google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

gemma_2b = Replicate(
    model        = 'google-deepmind/gemma-2b-it:dff94eaf770e1fc211e425a50b51baa8e4cac6c39ef074681f9e39d778773626',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}

)

phi = Replicate(
    model        = 'lucataco/phi-2:740618b0c24c0ea4ce5f49fcfef02fcd0bdd6a9f1b0c5e7c02ad78e9b3b190a6',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

yi_6b = Replicate(
    model        = '01-ai/yi-6b:d302e64fad6b4d85d47b3d1ed569b06107504f5717ee1ec12136987bec1e94f1',
    model_kwargs = {"temperature": 0.95, "top_p": 1, "max_new_tokens":512}
)

current_model = phixtral_7b
current_model('Hello. How are you?')

"Hello! I'm doing well, thank you. How can I assist you today?"

In [23]:
task = tasks['lbpp/15']
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

prompt_prefix = '''1. You are an expert Python programmer, and here is your task: {}.
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tests:

Test cases:

'''
print(prompt_prefix.format(task['prompt']) + task['test'])

1. You are an expert Python programmer, and here is your task: Write a python function "def bonus_calculation(employees: pd.DataFrame,
bonus_categories: Dict[str,float], additional_bonus: float) -> Dict[int,int]" that takes as input
the dataframe "employees", containing the following columns: employee_id, employee_name,
salary, total_in_sales. Additionally, the function takes a dictionary
containing the bonus categories ("Low" if total in sales is less than 10000, "Medium" if
total in sales is between 10000 and 20000 and "High" if total in sales is higher than
20000) with the correspondent bonus percentage that should be applied on each employee's
salary. Lastly, the additional_bonus is a percentage of the base salary (before the first bonus is applied) that should be added on top of the
regular bonus, but only for those who achieved the "High" bonus category. All the percentages
are in the decimal form, e.g. 0.10 for 10%.
Return a dictionary containing employee_id and the calculated b

In [24]:
#model_nickname = 'llama3_8B'
#model_nickname = 'mistral_7B'
#model_nickname = 'gemma_7B'
#model_nickname = 'phixtral_8B'
#model_nickname = 'gemma_2B'
#model_nickname = 'solar_10B'
model_nickname = 'phixtral'
#model_nickname = 'qwen_1_5_7B'
#model_nickname = 'mamba2'
#model_nickname = 'yi_6b'
#model_nickname = 'phi'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'logs/phixtral_samples_20250124_235230_6581.jsonl'

In [25]:
num_samples_per_task = 1
completions = []
is_full_func = []
print(f'Model: {model_nickname}\n')

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == 'lbpp/5':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body['prompt']) + task_body['test']
        try:
            completion  = current_model( full_prompt )
        except Exception as e:
            completion = f"Error generating completion:\n{e}"
        
        completion = remove_triple_backticks_at_edges(completion.strip())
        is_full = is_full_function(completion)
        is_full_func.append(is_full)
        if is_full:
            completion = add_import(completion)
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

Model: phixtral

lbpp/0
Prompt:
1. You are an expert Python programmer, and here is your task: Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You may assume that the input array has at least one row and one column..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code 

Prompt:
1. You are an expert Python programmer, and here is your task: Write a NamedTuple "Products" which contains four fields: name (str), category (str), price (float) and quantity (float). Then in the same code block write a function "def apply_discount(order_list: List['Products'], category: str, discount: float) -> float" that applies a discount to all products that fall within the given category in the order_list and returns the total order price. Write it in Python..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must sat

Prompt:
1. You are an expert Python programmer, and here is your task: Write a function "def at_least_this_fast(input_list: List[Tuple[float, float]]) -> float" that, given a list of tuples containing numeric times and locations on a one dimensional space sorted by time,
outputs a float representing the fastest speed of the traveller. Assume the speed between two neighbor points is constant. Write it in Python..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tests:

Test cases:



assert at_least_this_fast([(0,

Prompt:
1. You are an expert Python programmer, and here is your task: Write a function “def average_salary_by_department(df: pd.DataFrame) -> List[Tuple[str, float]]” that returns the average salary of the employees stratified by department. The df dataframe has the following columns: "employee_ID", "salary", "department". Write it in Python. Return the results sorted by average salary.
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tests:

Test cases:

import pandas as pd

df1 = pd.DataFrame(
    {
        "e

Prompt:
1. You are an expert Python programmer, and here is your task: Given a binary search tree and a range of integers represented by a tuple `(start, end)`, write a Python program to find all values in the binary search tree that fall within the range. The range is inclusive of the start and end values. Return them in a set. The program should also define a TreeNode class that represents the binary search tree. The TreeNode class should contain a constructor  __init__(self, x: int) that takes in an integer value that represents the value of the node. The TreeNode class should also have a function with signature add_node(self, x: int) that, when called on the root of the tree, places a new TreeNode at the appropriate spot in the tree with the value x..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should

...

Prompt:
1. You are an expert Python programmer, and here is your task: Write a class TreeNode with 3 attributes: val (int), left (Optional[TreeNode]), and right (Optional[TreeNode]).
Then write a python function to swap node values of the ith node at the kth level from the top and the ith node and kth level
from the bottom of a binary tree. Note that the height of a leaf node should be 0..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tests:

Test cases:

def inorderTraversal(root: TreeNode) -> list[int]:
    

Prompt:
1. You are an expert Python programmer, and here is your task: You are given a list of salaries, a mapping between marginal tax rate and the income at which that marginal tax rate applies, a maximum marginal tax rate, and a target tax revenue. You can not increase any of the marginal tax rates beyond the rate of the maximum marginal tax rate. You should only increase a marginal tax rate under one of the following conditions: 
1) It is the highest tax bracket 
2) All of the higher tax brackets have already reached the maximum marginal tax rate but the target tax revenue has not been reached. 
Given these constraints write a python function `def get_new_marginal_tax_rates(salaries: list[float], marginal_taxes: dict[float, float], max_tax_rate: float, target_revenue: float) -> dict[float, float]` that computes an array of marginal tax rates that accomplishes the target tax revenue..
2. Your output must include only the correct import statements, the function with proper Python cod

Prompt:
1. You are an expert Python programmer, and here is your task: Create a Python function `total_production(production: pd.DataFrame) -> List[Tuple[float, float, str]]`. Each row of `production` is a tuple representing a production line with values `<day, line, gross_production>`. Calculate the net production per line (gross production of the line * 80% yield). If the net production over a period is < 2000, discard it (set to zero) and reprocess the line. If the net production is between 2000 and 5000, quarantine products of the line unless the line's median net production is < 400. When it is < 400, discard the products. For discarded products, prepend "W_" to the line's name. There's also a special case where we want to combine `"Line A"` and `"Line B"` into `"Line A+B"`. Output a vector of tuples with `<total net production, total quarantined production, line names>` sorted by line names.

E.g. `(0,3000,'Line A+B'),(8000,0,'Line D'),(0,3000,'W_Line E')`..
2. Your output must i

Prompt:
1. You are an expert Python programmer, and here is your task: Write a python function "def total_sales(file_path: str) -> float" that access the daily sales dataset in a csv file, which contains information about unit price and quantity, and calculates the total revenue. The file contains the following columns: "order_id", "product_id", "unit_price" and "quantity". Include error handling, so it can detect if there's any problem with the imported data..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tes

Prompt:
1. You are an expert Python programmer, and here is your task: Write a python function "production_cost(raw_material_cost: Dict[str,float], raw_material_usage: Dict[str,float], daily_production: List[int]) -> float" function that takes as input two dictionaries: one containing
the cost of each raw material and the other containing the number of each
raw material that are used to produce one unit. Additionally, the function takes a list
containing the numbers of units produced each day. The function should
return the cost of producing all the units..
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with n

In [12]:
# if the above cell has not been run
#file_name = 'logs2/solar_10B_samples_20241127_232205_5243.jsonl'

In [26]:
# count how many full funcs were generated vs. only func bodies without func signatures
from collections import Counter
Counter(is_full_func)

Counter({True: 152, False: 10})

In [27]:
evaluate_functional_correctness(file_name, k=[1], mode='lbpp')

Reading samples...


162it [00:00, 29729.92it/s]


Running test suites...


100%|██████████████████████████████████████████████████████████████████| 162/162 [00:11<00:00, 13.99it/s]


Writing results to logs/phixtral_samples_20250124_235230_6581.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████████████████| 162/162 [00:00<00:00, 59187.91it/s]


{'pass@1': 0.04938271604938271}

## Getting Model Temperature and Top_p
According to [Mistral docs](https://docs.mistral.ai/api/#tag/chat/operation/chat_completion_v1_chat_completions_post):
* Default top_p is 1
* To find out about the default temperature, one neexs to call the `/models` endpoint to retrieve the appropriate value.

In [70]:
# calling the /models endpoint
import requests

api_key = os.environ["MISTRAL_API_KEY"]

# Correct API endpoint
url = "https://api.mistral.ai/v1/models"

# Headers with the correct authorization format
headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for HTTP 4xx/5xx codes
    print("Response:")
    print(response.json())
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


Response:
{'object': 'list', 'data': [{'id': 'ministral-3b-2410', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-latest'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-3b-latest', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-2410'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-8b-2410', 'object': 'model', 'cr

In [72]:
response.json()

{'object': 'list',
 'data': [{'id': 'ministral-3b-2410',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-latest'],
   'deprecation': None,
   'default_model_temperature': 0.3,
   'type': 'base'},
  {'id': 'ministral-3b-latest',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-2410'],
   'deprecation': None,
   'default_mode

In [79]:
for i in response.json()['data']:
    print(f"Model: {i['id']}. Temperature: {i['default_model_temperature']}")

Model: ministral-3b-2410. Temperature: 0.3
Model: ministral-3b-latest. Temperature: 0.3
Model: ministral-8b-2410. Temperature: 0.3
Model: ministral-8b-latest. Temperature: 0.3
Model: open-mistral-7b. Temperature: 0.7
Model: mistral-tiny. Temperature: 0.7
Model: mistral-tiny-2312. Temperature: 0.7
Model: open-mistral-nemo. Temperature: 0.3
Model: open-mistral-nemo-2407. Temperature: 0.3
Model: mistral-tiny-2407. Temperature: 0.3
Model: mistral-tiny-latest. Temperature: 0.3
Model: open-mixtral-8x7b. Temperature: 0.7
Model: mistral-small. Temperature: 0.7
Model: mistral-small-2312. Temperature: 0.7
Model: open-mixtral-8x22b. Temperature: 0.7
Model: open-mixtral-8x22b-2404. Temperature: 0.7
Model: mistral-small-2402. Temperature: 0.7
Model: mistral-small-2409. Temperature: 0.7
Model: mistral-small-latest. Temperature: 0.7
Model: mistral-medium-2312. Temperature: 0.7
Model: mistral-medium. Temperature: 0.7
Model: mistral-medium-latest. Temperature: 0.7
Model: mistral-large-2402. Temperature